<a href="https://colab.research.google.com/github/shiehn/dawnet-remotes/blob/main/DAWNet_Remote_Style_Transfer_AudioLDM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Check GPU

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [1]:
#@title Install AudioLDM2 Dependencies
import subprocess, os, sys, ipykernel

!pip install -q condacolab
import condacolab
condacolab.install()

res = subprocess.run(['conda', 'create', '-n' 'audioldm', 'python=3.8'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(res)

res = subprocess.run(['conda', 'activate', 'audioldm'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(res)

res = subprocess.run(['pip', 'install', 'audioldm'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(res)

# res = subprocess.run(['pip', 'install', 'git+https://github.com/haoheliu/AudioLDM2.git'], stdout=subprocess.PIPE).stdout.decode('utf-8')
# print(res)

#gitclone('https://github.com/haoheliu/AudioLDM')

!pip uninstall cffi -y
!pip install cffi

print('install complete')








✨🍰✨ Everything looks OK!
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local/envs/audioldm

  added / updated specs:
    - python=3.8


The following NEW packages will be INSTALLED:

  _libgcc_mutex      conda-forge/linux-64::_libgcc_mutex-0.1-conda_forge 
  _openmp_mutex      conda-forge/linux-64::_openmp_mutex-4.5-2_gnu 
  bzip2              conda-forge/linux-64::bzip2-1.0.8-hd590300_5 
  ca-certificates    conda-forge/linux-64::ca-certificates-2023.11.17-hbcca054_0 
  ld_impl_linux-64   conda-forge/linux-64::ld_impl_linux-64-2.40-h41732ed_0 
  libffi             conda-forge/linux-64::libffi-3.4.2-h7f98852_5 
  libgcc-ng          conda-forge/linux-64::libgcc-ng-13.2.0-h807b86a_3 
  libgomp            conda-forge/linux-64::libgomp-13.2.0-h807b86a_3 
  libnsl             conda-forge/linux-64::libnsl-2.0.1-hd590300_0 
  libsqlite          conda-forge/linux-64::libsqlite-3.44.2-h2797004_0 
  libuuid            conda-forge/linux-64::libuuid-2.38

install complete


In [3]:
#@title Generate Audio


!apt-get install ffmpeg

try:
    import dawnet_client
except ImportError:
  !pip install dawnet-client --upgrade


import dawnet_client.core as dawnet
from dawnet_client import DAWNetFilePath, utils

import subprocess, os, sys, ipykernel
import random
import shutil

placeholder_txt = "Enter the token generated by the DAWNet plugin"
DAWNET_TOKEN = "895f7fef-22de-4ce7-848b-48094c42a186" #@param {type:"string"}
dawnet_token = DAWNET_TOKEN

if dawnet_token is None or dawnet_token == "" or dawnet_token == placeholder_txt:
  print("ERROR: The token provided is not valid.")
  exit()


# model_path = os.path.join(
#             os.path.expanduser("~"),
#             ".cache/audioldm/audioldm-l-full.ckpt",
#         )

# print('MODEL EXISTS:', str(os.path.exists(model_path)))




outputs_per_input = 3
batch_size = 1
candidates = 4
transfer_style = 'Funky and dark'
transfer_strength = '0.5'
output_dir = '/transfer_output'



def ensure_empty_directory(output_dir):
    # Check if the directory exists
    if os.path.exists(output_dir):
        # Delete all contents of the directory
        shutil.rmtree(output_dir)

    # Create the directory (works regardless of whether it was just deleted or never existed)
    os.makedirs(output_dir)

def largest_perfect_multiple(audio_len):
    return audio_len - (audio_len % 2.5)


import os

def collect_outputs(input_file, output_dir):
    # Extract the file name without extension
    file_name = os.path.basename(input_file)
    file_name_without_ext = os.path.splitext(file_name)[0]

    # Create the new file path
    new_file_path = os.path.join(output_dir, "transfer", file_name_without_ext)

    # List of audio file extensions
    audio_extensions = ['.wav', '.aiff', '.aif', '.mp3']

    # Assuming the function does not actually transfer or create files,
    # we simulate by listing files in the given directory if they exist.
    file_paths = []
    if os.path.exists(new_file_path):
        for file in os.listdir(new_file_path):
            if os.path.splitext(file)[1].lower() in audio_extensions:
                # Check for white spaces in the file name
                if ' ' in file:
                    # Replace white spaces with underscores
                    new_file_name = file.replace(' ', '_')
                    old_file_path = os.path.join(new_file_path, file)
                    new_file_path_full = os.path.join(new_file_path, new_file_name)
                    # Rename the file
                    os.rename(old_file_path, new_file_path_full)
                    file = new_file_name

                file_paths.append(os.path.join(new_file_path, file))

    return file_paths


async def dawnet_func(input_file: DAWNetFilePath):

  try:
    ensure_empty_directory(output_dir)

    seed = str(random.randint(1, 1000))

    audio_len = utils.get_audio_length(input_file)

    audio_len = largest_perfect_multiple(audio_len)

    print("audio_length: " + str(audio_len))

    bpm_str = str(round(dawnet.get_daw_bpm()))
    print("bpm: " + bpm_str)

    print('INPUT_PATH: ' + str(input_file))
    print('OUTPUT_DIR: ' + str(output_dir))
    print('AUDIO_LEN: ' + str(audio_len))
    print('SEED: ' + seed)
    print('TRANSFER_STYLE: ' + transfer_style)
    print('BATCH SIZE: ' + str(batch_size))
    print('CANDIDATES: ' + str(candidates))

    print('RUN-START')

    #audioldm-l-full
    cmd = "audioldm --mode 'transfer' --file_path '{}' --save_path '{}' -t '{}' --transfer_strength {} --duration {} --seed {} --n_candidate_gen_per_text {} --batchsize {} --model_name audioldm-l-full".format(input_file, output_dir, transfer_style, transfer_strength, audio_len, seed, candidates, batch_size)

    !{cmd}

    print(res)

    outputs = collect_outputs(input_file, output_dir)
    for output in outputs:
      print("Adding output: " + str(output))
      await dawnet.output().add_file(output)

    await dawnet.output().add_message("Returning the input file")
    await dawnet.output().send()

    return True

  except Exception as e:
    print(f"Error in arbitrary_method: {e}")
    return False


dawnet.set_token(token=dawnet_token)
dawnet.set_name("AudioLDM2 - Style Transfer")
dawnet.set_description("This is a template for creating a DAWNet Remote")
dawnet.register_method(dawnet_func)

dawnet.connect_to_server()



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


2024-01-09 07:02:49,404 - websockets.client - DEBUG - > TEXT '{"token": "895f7fef-22de-4ce7-848b-48094c42a186... "data": {"status": 1}}' [92 bytes]
2024-01-09 07:02:49,408 - DN_CLIENT - INFO - Processing event: {'dn_msg_stage': 'CLIENT_REG_METHOD', 'dn_msg': 'Registered method: dawnet_func'}
2024-01-09 07:02:49,410 - websockets.client - DEBUG - > TEXT '{"token": "895f7fef-22de-4ce7-848b-48094c42a186... "data": {"status": 1}}' [92 bytes]
2024-01-09 07:02:49,414 - websockets.client - DEBUG - > TEXT '{"token": "895f7fef-22de-4ce7-848b-48094c42a186...", "version": "0.0.0"}}' [368 bytes]
2024-01-09 07:02:49,416 - DN_CLIENT - INFO - Processing event: {'dn_msg_stage': 'CLIENT_REG_CONTRACT', 'dn_msg': 'Sent contract for registration. Token: 895f7fef-22de-4ce7-848b-48094c42a186'}
2024-01-09 07:02:49,417 - websockets.client - DEBUG - > TEXT '{"token": "895f7fef-22de-4ce7-848b-48094c42a186... "data": {"status": 1}}' [92 bytes]
2024-01-09 07:02:49,421 - dawnet_client.core - INFO - Created a tempor

audio_length: 5.0
bpm: 170
INPUT_PATH: /tmp/tmpuli0bltc/resampled/section_two-hi-0-7db83d0b-d5ce-4e4b-a028-6e042383a762.wav
OUTPUT_DIR: /transfer_output
AUDIO_LEN: 5.0
SEED: 628
TRANSFER_STYLE: Funky and dark
BATCH SIZE: 1
CANDIDATES: 4
RUN-START
Load AudioLDM: %s audioldm-l-full
DiffusionWrapper has 739.14 M params.
/usr/local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/usr/local/lib/python3.10/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)
/usr/local/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be

2024-01-09 07:04:46,643 - DN_CLIENT - INFO - Processing event: {'dn_msg_stage': 'CLIENT_CONVERT_UPLOAD', 'dn_msg': '/transfer_output/transfer/section_two-hi-0-7db83d0b-d5ce-4e4b-a028-6e042383a762/resampled/09_01_2024_07_04_43_Funky and dark_0.wav'}
2024-01-09 07:04:46,652 - websockets.client - DEBUG - < PING '\x12OFF' [text, 4 bytes]
2024-01-09 07:04:46,656 - websockets.client - DEBUG - > PONG '\x12OFF' [text, 4 bytes]
2024-01-09 07:04:46,657 - websockets.client - DEBUG - % sending keepalive ping
2024-01-09 07:04:46,658 - websockets.client - DEBUG - > PING af ae 7d eb [binary, 4 bytes]
2024-01-09 07:04:46,701 - websockets.client - DEBUG - < PONG af ae 7d eb [binary, 4 bytes]
2024-01-09 07:04:46,702 - websockets.client - DEBUG - % received keepalive pong
2024-01-09 07:04:47,124 - DN_CLIENT - INFO - Processing event: {'dn_msg_stage': 'CLIENT_UPLOAD_ASSET', 'dn_msg': 'https://storage.googleapis.com/byoc-file-transfer/09_01_2024_07_04_43_Funky and dark_0.wav'}
2024-01-09 07:04:47,124 - web

KeyboardInterrupt: 